In [13]:
# -*- coding:utf-8 -*-
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import HumanMessage
import time, os
import api_key as api
import common_tools as common_tools
from PIL import Image
from io import BytesIO
import base64

api_key = api.openai_api_key()
# 将API Key保存为环境变量
os.environ["OPENAI_API_KEY"] = api_key
model_name='gpt-4o-mini'
# model_name='gpt-4o'

def image_summarize(img_base64, prompt):
    chat = ChatOpenAI(model_name=model_name)  # 使用支持图像的模型
    message = HumanMessage(
        content=[
            {"type": "text", "text": prompt},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"}
            }
        ]
    )
    response = chat([message])
    return response.content

def convert_to_base64(pil_image):
    buffered = BytesIO()
    pil_image.save(buffered, format="png")
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
#     img_str = resize_base64_image(img_str, size=(960, 540)) 
    return img_str

def chat_with_iamge():
    pil_image = Image.open("/Users/Daglas/Desktop/截屏2024-07-20 22.50.35.png")  # 打开实际的图像文件
    img_base64 = convert_to_base64(pil_image) 
    output = image_summarize(img_base64, "将以下图片中的公式转为LaTeX 语法，并以markdown格式打印出来")
    print(output) 

start_time = time.time()
print('waiting...\n')
chat_with_iamge()
end_time = time.time()
print('Time Used: ' + str((end_time - start_time)/60) + 'min')

waiting...



以下是图片中公式的 LaTeX 语法：

```latex
PE_{(pos,2i)} = \sin\left(\frac{pos}{10000^{\frac{2i}{d_{\text{model}}}}}\right)

PE_{(pos,2i+1)} = \cos\left(\frac{pos}{10000^{\frac{2i}{d_{\text{model}}}}}\right)
```

Time Used: 0.12784245411554973min
